In [1]:
import sys
sys.path.append("../")

In [2]:
from datasets.vocabulary import BertVocab

In [3]:
class config():
    def __init__(self):
        self.vocab_path = "../data/uncased_L-12_H-768_A-12/vocab.txt"

In [4]:
conf = config()

In [5]:
import tqdm

In [6]:
from collections import Counter

In [7]:
bert_counter = Counter()

In [8]:
print("Using Bert Vocab")
counter = Counter()
with open("../data/wikitext2/all.txt") as text_file:
    for line in tqdm.tqdm(text_file):
        if isinstance(line, list):
            words = line
        else:
            words = line.replace("\n", "").replace("\t", "").split()

        for word in words:
            counter[word] += 1

text_file.close()

Using Bert Vocab


98856it [00:01, 68669.27it/s]


In [9]:
import numpy as np

In [10]:
bert_counter = Counter()
word_list = []
with open("../data/uncased_L-12_H-768_A-12/vocab.txt") as bert_vocab:

    for word in tqdm.tqdm(bert_vocab):
        word = word.replace("\n", "").replace("\t", "")
        bert_counter[word] = counter[word] if word in counter else 1
        word_list.append(word)


30522it [00:00, 157945.80it/s]


In [11]:
import datasets

In [12]:
class config():
    def __init__(self):
        self.vocab = "bert-google"
        self.vocab_path = "../data/wikitext2/all.txt"
        self.bert_google_vocab = "../data/uncased_L-12_H-768_A-12/vocab.txt"
        self.vocab_max_size = None
        self.vocab_min_frequency = 1
        self.dataset = "wikitext2"
        self.seq_len = 40
        self.on_memory = True
        self.corpus_lines = None
        self.train_dataset = "../data/wikitext2/test_data_single_sentence.txt"
        self.encoding = "utf-8"
        self.batch_size = 1
        self.num_workers = 1

In [13]:
conf = config()

In [14]:
vocab = BertVocab(conf)

Using Bert Vocab


98856it [00:00, 115697.96it/s]
30522it [00:00, 983691.12it/s]


In [15]:
# load the dataset specified with --dataset_name & get data loaders
train_dataset = datasets.get(dataset_name="wikitext2")(config=conf, vocab=vocab)

train_loader = train_dataset.get_data_loader()

Loading Dataset: 9961it [00:00, 14818.51it/s]


In [16]:
data = next(iter(train_loader))

In [17]:
import models

In [18]:
conf.hidden_features = 768
conf.layers = 12
conf.heads = 12
conf.device = "cpu"
conf.dropout = 0.1
conf.train = True
conf.lr = 1e-3
conf.adam_beta1=0.999
conf.adam_beta2=0.999
conf.adam_weight_decay = 0.01
conf.warmup_steps =1000

In [19]:
model = models.get(model_name="BERTLM")(config=conf, vocab_size=len(vocab))

In [20]:
import torch
import torch.nn as nn

### Loading TF - BERT FTW!!!

In [21]:
pt_model = torch.load("models/_checkpoints/wikitext2/BERTLM-latest.pth")

In [22]:
from copy import deepcopy

In [23]:
model_copy = deepcopy(model)

In [24]:
new_model = model.load_state_dict(pt_model['model_state_dict'])

### Comparing with Hugging Face :)

In [25]:
model.bert.embedding.token.weight

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)

In [26]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
hf_model = BertModel.from_pretrained("bert-base-uncased")

2022-12-31 19:32:59.260833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

In [91]:
from transformers import BertForMaskedLM
ml_model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [99]:
ml_model.

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [98]:
model

BERTLM(
  (bert): BERT(
    (embedding): BERTEmbedding(
      (token): TokenEmbedding(30522, 768, padding_idx=0)
      (position): PositionalEmbedding()
      (segment): SegmentEmbedding(2, 768, padding_idx=0)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer_blocks): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadedAttention(
          (linear_layers): ModuleList(
            (0): Linear(in_features=768, out_features=768, bias=True)
            (1): Linear(in_features=768, out_features=768, bias=True)
            (2): Linear(in_features=768, out_features=768, bias=True)
          )
          (output_linear): Linear(in_features=768, out_features=768, bias=True)
          (attention): Attention()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=768, out_features=3072, bias=True)
          (w_2): Linear(in_features=3072, out_features=768, bias=True)

In [97]:
ml_model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [32]:
hf_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [44]:
model.bert.embedding.position.pe.shape

torch.Size([1, 512, 768])

In [53]:
model

BERTLM(
  (bert): BERT(
    (embedding): BERTEmbedding(
      (token): TokenEmbedding(30522, 768, padding_idx=0)
      (position): PositionalEmbedding()
      (segment): SegmentEmbedding(2, 768, padding_idx=0)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer_blocks): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadedAttention(
          (linear_layers): ModuleList(
            (0): Linear(in_features=768, out_features=768, bias=True)
            (1): Linear(in_features=768, out_features=768, bias=True)
            (2): Linear(in_features=768, out_features=768, bias=True)
          )
          (output_linear): Linear(in_features=768, out_features=768, bias=True)
          (attention): Attention()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=768, out_features=3072, bias=True)
          (w_2): Linear(in_features=3072, out_features=768, bias=True)

In [52]:
hf_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

### Position Embeddings

In [54]:
assert (model.bert.embedding.position.pe == hf_model.embeddings.position_embeddings).all()

AttributeError: 'bool' object has no attribute 'all'

In [62]:
hf_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [58]:
assert (model.bert.embedding.token.weight == hf_model.embeddings.word_embeddings.weight).all()

In [61]:
assert (model.bert.embedding.segment.weight == hf_model.embeddings.token_type_embeddings.weight).all()

In [ ]:
assert (model.bert.transformer_blocks[0] == hf_model.encoder.layer[0])

In [77]:
mapping = {
    'attention.self.query.weight':'attention.linear_layers.0.weight',
    'attention.self.query.bias':'attention.linear_layers.0.bias',
    'attention.self.key.weight':'attention.linear_layers.1.weight',
    'attention.self.key.bias':'attention.linear_layers.1.bias',
    'attention.self.value.weight':'attention.linear_layers.2.weight',
    'attention.self.value.bias':'attention.linear_layers.2.bias',
    'attention.output.dense.weight':'attention.output_linear.weight',
    'attention.output.dense.bias':'attention.output_linear.bias',
    'attention.output.LayerNorm.weight':'input_sublayer.norm.a_2',
    'attention.output.LayerNorm.bias': 'input_sublayer.norm.b_2',
    'intermediate.dense.weight':'feed_forward.w_1.weight',
    'intermediate.dense.bias':'feed_forward.w_1.bias',
    'output.dense.weight':'feed_forward.w_2.weight',
    'output.dense.bias':'feed_forward.w_2.bias',
    'output.LayerNorm.weight':'output_sublayer.norm.a_2',
    'output.LayerNorm.bias':'output_sublayer.norm.b_2',
}

In [85]:
for layer in range(12):
    param_dict = {}
    for name, param in model.bert.transformer_blocks[layer].named_parameters():
        param_dict[name] = param
    for name, param in hf_model.encoder.layer[layer].named_parameters():
        assert (param == param_dict[mapping[name]]).all()
        print(name, mapping[name])
    print(layer, " ========= \n")

attention.self.query.weight attention.linear_layers.0.weight
attention.self.query.bias attention.linear_layers.0.bias
attention.self.key.weight attention.linear_layers.1.weight
attention.self.key.bias attention.linear_layers.1.bias
attention.self.value.weight attention.linear_layers.2.weight
attention.self.value.bias attention.linear_layers.2.bias
attention.output.dense.weight attention.output_linear.weight
attention.output.dense.bias attention.output_linear.bias
attention.output.LayerNorm.weight input_sublayer.norm.a_2
attention.output.LayerNorm.bias input_sublayer.norm.b_2
intermediate.dense.weight feed_forward.w_1.weight
intermediate.dense.bias feed_forward.w_1.bias
output.dense.weight feed_forward.w_2.weight
output.dense.bias feed_forward.w_2.bias
output.LayerNorm.weight output_sublayer.norm.a_2
output.LayerNorm.bias output_sublayer.norm.b_2
0  ========= 

attention.self.query.weight attention.linear_layers.0.weight
attention.self.query.bias attention.linear_layers.0.bias
attention

In [93]:
model.mask_lm.linear.weight

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)

In [96]:
model.mask_lm

MaskedLanguageModel(
  (linear): Linear(in_features=768, out_features=30522, bias=True)
  (softmax): LogSoftmax(dim=-1)
)

In [95]:
for name, param in ml_model.cls.named_parameters():
    print(name, param.shape)

predictions.bias torch.Size([30522])
predictions.transform.dense.weight torch.Size([768, 768])
predictions.transform.dense.bias torch.Size([768])
predictions.transform.LayerNorm.weight torch.Size([768])
predictions.transform.LayerNorm.bias torch.Size([768])
predictions.decoder.weight torch.Size([30522, 768])


In [90]:
hf_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          